In [ ]:
########################################### STEP 1 -  의존성 로드 (초기에만 수행)######################################################
import numpy as np
import pandas as pd
import os
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.impute import KNNImputer
import seaborn as sns # seaborn ref : https://greeksharifa.github.io/machine_learning/2019/12/05/Seaborn-Module/
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.svm import SVC
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import statsmodels.api as sm;
from patsy import dmatrices
from statsmodels.stats.outliers_influence import variance_inflation_factor
%matplotlib inline
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 200)
########################################################################################################################################

In [ ]:
########################################### STEP 2 - 데이터 임포팅 (초기에만 수행) ######################################################
df = pd.read_csv("2018_SEL_FILLED.csv") ## 파일리딩!
#df.head()
########################################################################################################################################

In [ ]:
######################################### STEP 3 - missing value filling(초기에만 수행) ###############################################
#df=df.fillna(method='pad')

# 범주형,이산형 변수인 경우 K-NN 수행
# df = pd.read_excel('test.xlsx')
# imputer = KNNImputer(n_neighbors=11)
# data_filled = imputer.fit_transform(df)
# data_filled = data_filled.astype('int64')
# download = pd.DataFrame(data_filled).to_csv('filled.csv')

# 연속형 변수인 경우 평균화기법 수행
# df = pd.read_excel('test.xlsx')
# data_filled = df.where(pd.notnull(df), df.median(), axis='columns')
# download = pd.DataFrame(data_filled).to_csv('filled.csv')

#df=df.drop(df.columns[0], axis='columns') #unnamed drop 없을경우 주석 처리
df.isnull()
########################################################################################################################################

In [ ]:
######################################### STEP 4 - Data에 대한 구조와 분석 (반복)######################################################
df.hist(figsize=(20,20))#histogram plotting
#sns.paiplot(df)
df.head()
df['HE_DM']
########################################################################################################################################

In [ ]:
###################################### STEP 5 = 변수간 상관관계 및 다중공선성(VIF) 분석 (반복)#########################################

# 변수간 상관관계 분석 - 1 
t=df.corr(method='pearson') ## de1_pr 현재 당뇨병 유병 여부와 상관관계가 있는 변수들
t.HE_DM

In [ ]:
#변수간 상관관계 분석 - 2
sns.heatmap(df.corr(),cmap="YlGnBu")
t.head(200)

In [ ]:
#VIF 산출
df.dropna(inplace=True)
df['intercept'] = 1
lm = sm.OLS(df['HE_DM'], df)
results = lm.fit()
results.summary()

# dmatrix 사용을 위한 모든 변수에 대한 string +
cols = ''
for col in list(df):
    if(col=='sex'):
        cols = cols + str(col)
    else:
        cols = cols + ' + ' + str(col)
print(cols)

In [ ]:
# VIF 지수 산출 - 10 이상인 친구들     
y, X = dmatrices('HE_DM ~'+ cols , df, return_type = 'dataframe')
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns 
# ov_list=list(vif,list(vif['VIF Factor']>10))
# print(ov_list)
vif.head(200)
over_vif_vars = vif[vif['VIF Factor']>10]
over_vif_vars.head(200)
########################################################################################################################################

In [ ]:
########################################## STEP 6 - 변수의 RF 모델에서 나온 중요도 산출 (반복) ########################################

#인코딩
df.loc[df["HE_DM"] == 1,"HE_DM"] = 0
df.loc[df["HE_DM"] == 2,"HE_DM"] = 0
df.loc[df["HE_DM"] == 3,"HE_DM"] = 1
df['HE_DM'].head(100)


importance_list = []
y = df.HE_DM # y축 설정
x = df.drop(columns=["HE_DM"]) #결과축 삭제 for predictions
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42) 
rnd_clf = RandomForestClassifier(n_estimators=500,n_jobs=-1)
rnd_clf.fit(x_train,y_train)
for name, score in zip(df[:],rnd_clf.feature_importances_): ## RF 적 방법으로 변수 중요도 추출
    importance_list = importance_list + [name,score]
    print(name, round(score,6))
########################################################################################################################################

In [ ]:
############################### STEP 7 - step5, step 6의 결론을 토대로 변수 drop (반복)#########################################################
df = df.drop(['변수명'],axis='columns')
#df = df.drop(['DI4_pr','DM1_pr','DM2_pr','DM3_pr','DM4_pr','DJ2_pr','DJ4_pr','DC1_pr','DC2_pr','DC3_pr','DC3_pr','DC4_pr','DC5_pr','DC6_pr','DC7_pr','DF2_pr', 'DL1_pr', 'DJ0_pr', 'DJ6_pr', 'DH4_pr', 'DH3_pr', 'DH6_pr', 'DN1_pr', 'DK0_pr', 'DKNaN_pr', 'DK4_pr', 'LQ4_00', 'LQ1_sb', 'MH1_yr', 'MO1_wk','BD1','BS1_1','BS9_2','BE3_71','HE_mens','HE_prg','HE_DMfh1','HE_Usg','HE_Uket','HE_Ubld','HE_pef','HE_cough1'],axis='columns')
########################################################################################################################################

In [ ]:
###################################### STEP 8 - 파이프라이닝 - 스케일링 / 모델링 결정 (반복) ##########################################

## 데이터 분류 및 전처리
# 축 분리#te
Y = df.HE_DM # y축 설정
X = df.drop(columns=["HE_DM"]) #결과축 삭제 for predictions

# 사이킷런 셋 분류 함수 : http://blog.naver.com/PostView.nhn?blogId=siniphia&logNo=221396370872
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.30, random_state=42) ## test.size 0.2~0.25 설정할거임


pipe = Pipeline([('scaler', StandardScaler()), ('svc', SVC())])
pipe.fit(X_train,Y_train)
y_preds=pipe.predict(X_test)
print(pipe.score(X_test,Y_test))
plot_confusion_matrix(pipe,X_test,Y_test)
########################################################################################################################################